In [ ]:
!pip install degiro_connector pandas scipy sqlitedict polars yfinance
#!pip install selenium
!pip install --upgrade lxml


In [ ]:
from screener import Screener 
import logging
import sys
import pandas as pd

pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.1f}".format

logfile = "./outdegiro.log"
logging_level = logging.INFO
logging.basicConfig(
    level=logging_level,
    handlers=[logging.FileHandler(logfile, mode="a"), logging.StreamHandler(sys.stdout), ],
    format="%(asctime)s - %(name)s:%(filename)s:%(funcName)s:%(lineno)d - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

'''
logging.getLogger("yfinance").setLevel(logging.DEBUG)
logging.getLogger("requests_cache").setLevel(logging.DEBUG)
logging.getLogger("cachedYahooApi").setLevel(logging.DEBUG)

main_logger = logging.getLogger()
'''

df = Screener(None, None, None, None)


In [ ]:
import csv

if df is not None:
    ddf = df.copy()
     
    QS = 80.0  # score loic
    QSP = 90.0  # score loic + perf
    REV = 3   # croissance revenu
    EPS = 3   # croissance des profits nets
    MRG = 7   # marge
    SLD = 10  # en solde de x%
    LH = 100  # cours relatif entre le plus bas annuel et le plus haut [0,1]
    YLD = 10  # rendement dividende+prix
    PRX = 0   # croissance annuelle du prix de l'action, sans les dividendes
    MCAP = 5*10**7
    
    ddf = ddf[
        (ddf["qscore"] >= QS)
        & (ddf["qscorePerf"] >= QSP)
        & (ddf["REVPS5YGR"] >= REV)
        & (ddf["MARGIN5YR"] >= MRG)
        & (ddf["En Solde"] >= SLD)
        & (ddf["L%H"] <= LH)
        & (ddf["ChPctPrice5Y"] >= PRX)
        & (ddf["YLD+PRY"] >= YLD)
        & (ddf["MKTCAP.USD"] >= MCAP)
    ]
    
    # Removing banks, freight, holdings and mines
    ddf["keep"] = 1
    ddf.loc[(ddf["sector"].str.contains("Financial", case=False, regex=True) & ddf["industry"].str.contains("(?:bank|investment)", case=False, regex=True)), "keep"] = 0
    ddf.loc[(ddf["sector"].str.contains("Basic Materials", case=False, regex=True) & ddf["industry"].str.contains("Mining", case=False, regex=True)), "keep"] = 0
    ddf.loc[(ddf["sector"].str.contains("Transportation", case=False, regex=True) & ddf["industry"].str.contains("Freight", case=False, regex=True)), "keep"] = 0
    ddf.loc[ddf["name"].str.contains("holding", case=False, regex=True), "keep"] = 0
    ddf = ddf[ddf["keep"] == 1]

    ddf = ddf[
        [
            "isin", "sector", "country", "name", "industry", "qscore", "qscorePerf", "REVPS5YGR", "EPSTRENDGR", "MARGIN5YR", "PER",  "En Solde", 
            "L%H", "%M200D", "ChPctPrice5Y", "Rendement", "qMKTCAP.USD", "VOL10DAVG"
        ]
    ]
    
    print(ddf.shape)
    display(ddf)
    #print(ddf.dtypes, list(ddf.columns))
    #ddf.to_csv("screener5.csv", index=False, sep=str(";"), decimal=str(","), encoding="utf-8-sig", float_format="%.3f", quoting=csv.QUOTE_MINIMAL)
